# Housing Price Prediction Using Linear Regression Model

In [2]:
# !pip install dash

In [3]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
# Load the dataset
real_estate_data = pd.read_csv("Housing.csv") #replace the path with your path
real_estate_data.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [4]:
real_estate_data.columns

Index(['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'mainroad',
       'guestroom', 'basement', 'hotwaterheating', 'airconditioning',
       'parking', 'prefarea', 'furnishingstatus'],
      dtype='object')

In [5]:
# importing necessary libraries for Building linear regression model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Selecting features and target variable
features = ['area', 'bedrooms', 'bathrooms', 'parking']
target = 'price'

X = real_estate_data[features]
y = real_estate_data[target]

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model initialization
model = LinearRegression()

# Training the model
model.fit(X_train, y_train)

LinearRegression()

In [6]:
import joblib

# saving the model
joblib.dump(model, 'house_price_model.pkl')

['house_price_model.pkl']

In [7]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import joblib
import numpy as np

# Load the trained model
model = joblib.load('house_price_model.pkl')

# Initialize the Dash app
app = dash.Dash(__name__)

# Layout of the app
app.layout = html.Div([
    html.Div([
        html.H1("House Price Prediction", style={'text-align': 'center'}),
        
        html.Div([
            dcc.Input(id='area', type='number', placeholder='area',
                      style={'margin': '10px', 'padding': '10px'}),
            dcc.Input(id='bedrooms', type='number', placeholder='bedrooms',
                      style={'margin': '10px', 'padding': '10px'}),
            dcc.Input(id='bathrooms', type='number', placeholder='bathrooms',
                      style={'margin': '10px', 'padding': '10px'}),
            dcc.Input(id='parking', type='number', placeholder='parking',
                      style={'margin': '10px', 'padding': '10px'}),
            html.Button('Predict', id='predict-button', n_clicks=0,
                        style={'margin': '10px', 'padding': '10px', 'background-color': '#007BFF', 'color': 'white'}),
        ], style={'text-align': 'center'}),
        
        html.Div(id='prediction_output', style={'text-align': 'center', 'font-size': '20px', 'margin-top': '20px'})
    ], style={'width': '50%', 'margin': '0 auto', 'border': '2px solid #007BFF', 'padding': '20px', 'border-radius': '10px'})
])

# Callback to update the predicted price
@app.callback(
    Output('prediction_output', 'children'),
    Input('predict-button', 'n_clicks'),
    Input('area', 'value'),
    Input('bathrooms', 'value'),
    Input('bedrooms', 'value'),
    Input('parking', 'value')
)
def update_output(n_clicks, area, bathrooms, bedrooms, parking):
    if n_clicks > 0:
        # Check if all inputs are provided
        if area is not None and bathrooms is not None and bedrooms is not None and parking is not None:
            # Prepare the input for the model
            input_features = np.array([[area, bathrooms, bedrooms, parking]])
            # Make prediction
            predicted_price = model.predict(input_features)[0]
            return f'Predicted Price: Rs. {predicted_price:,.0f}' 
            
        else:
            return "Please fill in all fields."
    return ""

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)